# NAIVE BAYES FOR STOCKS

In [112]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime
import py_vollib.black_scholes.implied_volatility as iv
import warnings
warnings.filterwarnings("ignore")

In [269]:
#ticker = input("Type in the stock ticker")
#start_date = input("Type in a start date (Ex.: 2010-01-01)")
start_date = input('Start date (ex.: 2010-01-01): ')
ticker = input('Ticker: ')
ticker = ticker+'.SA'

end_date = datetime.today().strftime('%Y-%m-%d')

data_downloaded = yf.download(ticker, start=start_date)

data = data_downloaded

Start date (ex.: 2010-01-01): 2010-01-01
Ticker: BBAS3
[*********************100%***********************]  1 of 1 completed


In [270]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,29.799999,30.000000,29.700001,29.900000,12.371157,3624700
2010-01-05,30.100000,30.100000,29.190001,29.600000,12.247033,3064900
2010-01-06,29.490000,30.000000,29.420000,29.639999,12.263578,2733100
2010-01-07,29.440001,29.709999,29.410000,29.650000,12.267723,1542200
2010-01-08,29.700001,29.820000,29.469999,29.820000,12.338059,2038900
...,...,...,...,...,...,...
2023-07-07,49.040001,50.320000,49.040001,49.959999,49.959999,11539600
2023-07-10,49.990002,50.049999,48.790001,49.040001,49.040001,8903000
2023-07-11,49.000000,49.020000,47.880001,48.340000,48.340000,12336800


In [271]:
underlying_price = data['Close'][-1]

In [272]:
df = data[['Close','Volume', 'High']]
df

,Close,Volume,High
Date,,,
2010-01-04,29.900000,3624700,30.000000
2010-01-05,29.600000,3064900,30.100000
2010-01-06,29.639999,2733100,30.000000
2010-01-07,29.650000,1542200,29.709999
2010-01-08,29.820000,2038900,29.820000
...,...,...,...
2023-07-07,49.959999,11539600,50.320000
2023-07-10,49.040001,8903000,50.049999
2023-07-11,48.340000,12336800,49.020000


In [273]:
df['High Tomorrow']=df.High.shift(-1)
df

,Close,Volume,High,High Tomorrow
Date,,,,
2010-01-04,29.900000,3624700,30.000000,30.100000
2010-01-05,29.600000,3064900,30.100000,30.000000
2010-01-06,29.639999,2733100,30.000000,29.709999
2010-01-07,29.650000,1542200,29.709999,29.820000
2010-01-08,29.820000,2038900,29.820000,30.070000
...,...,...,...,...
2023-07-07,49.959999,11539600,50.320000,50.049999
2023-07-10,49.040001,8903000,50.049999,49.020000
2023-07-11,48.340000,12336800,49.020000,48.939999


In [274]:
df['High Tomorrow Change'] =((df['High Tomorrow']/df['Close'])-1)*100
df

,Close,Volume,High,High Tomorrow,High Tomorrow Change
Date,,,,,
2010-01-04,29.900000,3624700,30.000000,30.100000,0.668899
2010-01-05,29.600000,3064900,30.100000,30.000000,1.351350
2010-01-06,29.639999,2733100,30.000000,29.709999,0.236166
2010-01-07,29.650000,1542200,29.709999,29.820000,0.573356
2010-01-08,29.820000,2038900,29.820000,30.070000,0.838364
...,...,...,...,...,...
2023-07-07,49.959999,11539600,50.320000,50.049999,0.180144
2023-07-10,49.040001,8903000,50.049999,49.020000,-0.040784
2023-07-11,48.340000,12336800,49.020000,48.939999,1.241205


In [275]:
df['Result Tomorrow'] =df['High Tomorrow']-df['Close']
df['Tomorrow'] = (df['Result Tomorrow'] > 0).astype(int)
df

,Close,Volume,High,High Tomorrow,High Tomorrow Change,Result Tomorrow,Tomorrow
Date,,,,,,,
2010-01-04,29.900000,3624700,30.000000,30.100000,0.668899,0.200001,1
2010-01-05,29.600000,3064900,30.100000,30.000000,1.351350,0.400000,1
2010-01-06,29.639999,2733100,30.000000,29.709999,0.236166,0.070000,1
2010-01-07,29.650000,1542200,29.709999,29.820000,0.573356,0.170000,1
2010-01-08,29.820000,2038900,29.820000,30.070000,0.838364,0.250000,1
...,...,...,...,...,...,...,...
2023-07-07,49.959999,11539600,50.320000,50.049999,0.180144,0.090000,1
2023-07-10,49.040001,8903000,50.049999,49.020000,-0.040784,-0.020000,0
2023-07-11,48.340000,12336800,49.020000,48.939999,1.241205,0.599998,1


In [276]:
pct=df.Close.pct_change()
df['Price']=pct
df

,Close,Volume,High,High Tomorrow,High Tomorrow Change,Result Tomorrow,Tomorrow,Price
Date,,,,,,,,
2010-01-04,29.900000,3624700,30.000000,30.100000,0.668899,0.200001,1,NaN
2010-01-05,29.600000,3064900,30.100000,30.000000,1.351350,0.400000,1,-0.010033
2010-01-06,29.639999,2733100,30.000000,29.709999,0.236166,0.070000,1,0.001351
2010-01-07,29.650000,1542200,29.709999,29.820000,0.573356,0.170000,1,0.000337
2010-01-08,29.820000,2038900,29.820000,30.070000,0.838364,0.250000,1,0.005734
...,...,...,...,...,...,...,...,...
2023-07-07,49.959999,11539600,50.320000,50.049999,0.180144,0.090000,1,0.019384
2023-07-10,49.040001,8903000,50.049999,49.020000,-0.040784,-0.020000,0,-0.018415
2023-07-11,48.340000,12336800,49.020000,48.939999,1.241205,0.599998,1,-0.014274


In [277]:
vol=df.Volume.pct_change()
df['Vol']=vol
df

,Close,Volume,High,High Tomorrow,High Tomorrow Change,Result Tomorrow,Tomorrow,Price,Vol
Date,,,,,,,,,
2010-01-04,29.900000,3624700,30.000000,30.100000,0.668899,0.200001,1,NaN,NaN
2010-01-05,29.600000,3064900,30.100000,30.000000,1.351350,0.400000,1,-0.010033,-0.154440
2010-01-06,29.639999,2733100,30.000000,29.709999,0.236166,0.070000,1,0.001351,-0.108258
2010-01-07,29.650000,1542200,29.709999,29.820000,0.573356,0.170000,1,0.000337,-0.435732
2010-01-08,29.820000,2038900,29.820000,30.070000,0.838364,0.250000,1,0.005734,0.322072
...,...,...,...,...,...,...,...,...,...
2023-07-07,49.959999,11539600,50.320000,50.049999,0.180144,0.090000,1,0.019384,0.207324
2023-07-10,49.040001,8903000,50.049999,49.020000,-0.040784,-0.020000,0,-0.018415,-0.228483
2023-07-11,48.340000,12336800,49.020000,48.939999,1.241205,0.599998,1,-0.014274,0.385690


In [278]:
df2=df[['Price','Vol','Tomorrow', 'High Tomorrow Change']]
df2

,Price,Vol,Tomorrow,High Tomorrow Change
Date,,,,
2010-01-04,NaN,NaN,1,0.668899
2010-01-05,-0.010033,-0.154440,1,1.351350
2010-01-06,0.001351,-0.108258,1,0.236166
2010-01-07,0.000337,-0.435732,1,0.573356
2010-01-08,0.005734,0.322072,1,0.838364
...,...,...,...,...
2023-07-07,0.019384,0.207324,1,0.180144
2023-07-10,-0.018415,-0.228483,0,-0.040784
2023-07-11,-0.014274,0.385690,1,1.241205


In [279]:
df2=df2.dropna()
df2

,Price,Vol,Tomorrow,High Tomorrow Change
Date,,,,
2010-01-05,-0.010033,-0.154440,1,1.351350
2010-01-06,0.001351,-0.108258,1,0.236166
2010-01-07,0.000337,-0.435732,1,0.573356
2010-01-08,0.005734,0.322072,1,0.838364
2010-01-11,0.007713,0.183383,1,1.331120
...,...,...,...,...
2023-07-06,-0.014676,0.686845,1,2.672927
2023-07-07,0.019384,0.207324,1,0.180144
2023-07-10,-0.018415,-0.228483,0,-0.040784


In [280]:
df2['Price'] = (df2['Price'] > 0).astype(int)
df2['Vol'] = (df2['Vol'] > 0).astype(int)
df2

,Price,Vol,Tomorrow,High Tomorrow Change
Date,,,,
2010-01-05,0,0,1,1.351350
2010-01-06,1,0,1,0.236166
2010-01-07,1,0,1,0.573356
2010-01-08,1,1,1,0.838364
2010-01-11,1,1,1,1.331120
...,...,...,...,...
2023-07-06,0,1,1,2.672927
2023-07-07,1,1,1,0.180144
2023-07-10,0,0,0,-0.040784


In [281]:
combinacoes = df2.groupby(['Price', 'Vol', 'Tomorrow']).size().reset_index(name='Count')

# Calcular a probabilidade de cada combinação
total_combinacoes = combinacoes['Count'].sum()

combinacoes['Probability'] = combinacoes['Count'] / total_combinacoes

# Exibir as probabilidades de cada combinação
combinacoes

,Price,Vol,Tomorrow,Count,Probability
0,0,0,0,145,0.043232
1,0,0,1,755,0.225104
2,0,1,0,121,0.036076
3,0,1,1,678,0.202147
4,1,0,0,146,0.043530
5,1,0,1,679,0.202445
6,1,1,0,105,0.031306
7,1,1,1,725,0.216160


In [282]:
NN= combinacoes['Probability'][1]/(combinacoes['Probability'][1]+combinacoes['Probability'][0])*100
print('NN: ',NN)
NY= combinacoes['Probability'][3]/(combinacoes['Probability'][3]+combinacoes['Probability'][2])*100
print('NY: ',NY)
YN= combinacoes['Probability'][5]/(combinacoes['Probability'][5]+combinacoes['Probability'][4])*100
print('YN: ',YN)
YY= combinacoes['Probability'][7]/(combinacoes['Probability'][7]+combinacoes['Probability'][6])*100
print('YY: ',YY)

NN:  83.88888888888889
NY:  84.85607008760951
YN:  82.3030303030303
YY:  87.34939759036145


In [283]:
df3 = df2[(df2['Price'] == 1) & (df2['Vol'] == 0) & (df2['Tomorrow'] == 1)]
df3['High Tomorrow Change'].median()

1.446992989069007

In [284]:
def calculate_historical_volatility(prices, window):
    log_returns = np.log(prices / prices.shift(1))  # Calculate log returns
    historical_volatility = log_returns.rolling(window).std() * np.sqrt(252)  # Rolling standard deviation
    return historical_volatility

# Example usage
# Assuming you have a dataframe called 'df' with a column 'Close' representing the stock prices
prices = df['Close']
window_size = 30  # Define the window size (number of days)

historical_volatility = calculate_historical_volatility(prices, window_size)

# Print the historical volatility values
print('Hist.Vol: ',historical_volatility[-1])

Hist.Vol:  0.2760461302553121


In [286]:
option_price = float(input('Option price: '))
strike_price = float(input('Strike price: '))
time_to_maturity = int(input('Days to maturity: '))

time_to_maturity= time_to_maturity/365

risk_free_rate = 0.05  # Risk-free interest rate
flag=input('c or p: ')

implied_volatility = iv.implied_volatility(option_price, underlying_price, strike_price, time_to_maturity, risk_free_rate, flag)

# Print the implied volatility
print("Implied Volatility: ", implied_volatility)

Option price: 0.43
Strike price: 48.89
Days to maturity: 5
c or p: c
Implied Volatility:  0.32382042359838503
